### Exemplo de robô simples

Abaixo, um exemplo de loop de robô, que checa os preços a cada minutos e:
- Se a expectativa é de subida (+0,02%) e tem dinheiro em caixa, compra o equivalente a USD 10
- Se a expectativa é de queda (-0,02%) e tem a moeda em caixa, vende o equivalente a USD 10

In [ ]:
import pickle
import time
from simple_robot import feature_eng, api_get, api_post, get_result, compute_quantity, how_much_i_have
import numpy as np
import pandas as pd
from keras.models import load_model

# Carregando modelo criado no notebook anterior
model = pickle.load(open('model_lstm.pickle', 'rb'))
token = '4c9c24a24947fe3' #RAMster_Trade
ticker = 'ETHUSDT'
valor_compra_venda = 10

# Loop que checa o preço a cada minuto e toma as decisões de compra e venda.
while True:
    # Pegando o OHLC dos últimos 500 minutos
    df = api_post('cripto_quotation', {'token': token, 'ticker': ticker})
    
    # Features acrescentadas pelo grupo 2
    df['return'] = round(df['close'] / df['open'] - 1, 3)
    # Change Feature
    # Change of the price from previous day, absolute value
    df['change'] = (df.close - df.close.shift(1)).fillna(0)
    # Volatility Feature
    df['volatility'] = df.close.ewm(21).std()
    # Momentum
    df['momentum'] = momentum(df.close, 3)
    # RSI (Relative Strength Index)
    df['rsi'] = rsi(df)    
    # MACD - (Moving Average Convergence/Divergence)
    df['macd'], df['signal'] = macd(df)
    # Upper Band and Lower Band for Bollinger Bands
    df['upper_band'], df['lower_band'] = bollinger_bands(df)
    df.dropna(inplace=True)
    
    # Realizando a engenharia de features
    df = feature_eng(df)
    
    # Isolando a linha mais recente
    df_last = df.iloc[[np.argmax(df['time'])]]
    # utilizado para os cálculos abaixo, pois as chamadas são feitas por coluna. 
    # Ao converte para array numpy, o nome das colunas se perdem
    df_last_panda = df_last 
    # Precisamos converter para array numpy, pois precisamos adicionar uma dimensão "idle" 
    # para adequar ao modelo LSTM
    df_last = np.array(df_last) 
    #Adicionando a 3a dimensão ao dataframe, para poder executar a predição
    df_last = df_last.reshape(1,df_last.shape[0],df_last.shape[1]) 

    
    # Calculando tendência, baseada no modelo linear criado
    tendencia = model.predict(df_last)#.iloc[0]
    
    # A quantidade de cripto que será comprada/ vendida depende do valor_compra_venda e da cotação atual
    qtdade = compute_quantity(coin_value = df_last['close'], invest_value = valor_compra_venda, significant_digits = 2)
    
    # Print do datetime atual
    print('-------------------')
    now = pd.to_datetime('now')
    print(f'{now}')
    
    if tendencia > 0.02:
        # Modelo detectou uma tendência positiva
        print(f"Tendência positiva de {str(tendencia)}")
        
        # Verifica quanto dinheiro tem em caixa
        qtdade_money = how_much_i_have('money', token)
        
        if qtdade_money>0:
            # Se tem dinheiro, tenta comprar o equivalente a qtdade ou o máximo que o dinheiro permitir
            max_qtdade = compute_quantity(coin_value = df_last['close'], invest_value = qtdade_money, significant_digits = 2)
            qtdade = min(qtdade, max_qtdade)
            
            # Realizando a compra
            print(f'Comprando {str(qtdade)} {ticker}')
            api_post('buy', payload = {'token': token, 'ticker': ticker, 'quantity': qtdade})
        
    elif tendencia < -0.02:
        # Modelo detectou uma tendência negativa
        print(f"Tendência negativa de {str(tendencia)}")
        
        # Verifica quanto tem da moeda em caixa
        qtdade_coin = how_much_i_have(ticker, token)
        
        if qtdade_coin>0:
            # Se tenho a moeda, vou vender!
            qtdade = min(qtdade_coin, qtdade)
            print(f'Vendendo {str(qtdade)} {ticker}')
            api_post('sell', payload = {'token': token, 'ticker': ticker, 'quantity':qtdade})
    else:
        # Não faz nenhuma ação, espera próximo loop
        print(f"Tendência neutra de {str(tendencia)}. Nenhuma ação realizada")
    
    # Print do status após cada iteração
    print(api_post('status', payload = {'token': token}))
    time.sleep(60)